<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/notebooks/wxee_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q wxee

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 5.5 MB/s eta 0:00:00


# WXEE

here is the link to its [docs](https://wxee.readthedocs.io/en/latest/getting_started.html)

In [3]:
import ee
import wxee

ee.Authenticate()
wxee.Initialize(project='kavyajeetbora-ee')

In [4]:
admin = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
delhi = admin.filter(ee.Filter.eq('ADM1_NAME', 'Delhi'))

s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
geometry = delhi.geometry()

filtered_img = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',30))\
.filter(ee.Filter.date('2023-01-01','2023-02-01'))\
.filter(ee.Filter.bounds(geometry))

filtered_img = filtered_img.select('B1')

filtered_img.size().getInfo()

3

## Converting the image collection to xarray


When converting the image collection to an xarray dataset there might be an issue of maximum pixel request exceeding, here is the error that will be thrown:

```python
EEException: Total request size (1679993625 bytes) must be less than or equal to 50331648 bytes
```
The error you encountered is due to the size limitation of Earth Engine requests. When specifying a scale of 30 meters, the resulting request exceeds the allowed size.

To work around this limitation, consider the following approaches:

1. **Spatial Subsetting**:
    - Instead of processing the entire area of interest, consider subsetting your region into smaller tiles. Process each tile separately with the desired scale (e.g., 30 meters).
    - After processing, you can merge the results to obtain the complete xarray dataset.

2. **Downsampling**:
    - If the original imagery has a higher resolution (e.g., 10 meters), you can downsample it to 30 meters using resampling techniques (e.g., bilinear or nearest-neighbor).
    - Once downsampled, convert the resulting image to an xarray dataset.

3. **Aggregate Over Time**:
    - If you're working with a time series of imagery, aggregate the data over a specific time period (e.g., monthly or yearly) to reduce the request size.
    - Then convert the aggregated data to an xarray dataset.

Remember that Earth Engine has limitations on request size, so breaking down the task into smaller chunks or aggregating data can help overcome this issue.

**Note**:

The pixel limit set by Google Earth Engine applies to each individual image rather than the entire image collection. When you request data from Earth Engine, it processes each image separately. If an image exceeds the allowed size (e.g., due to a high-resolution dataset), you’ll encounter the error related to the request size

In [ ]:
image = filtered_img.first()
num_pixels = image.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=image.geometry(),
    scale=image.projection().nominalScale(),
    maxPixels=1e13  # Adjust this value based on your image size
).getInfo()

num_pixels

{'B1': 3348900}

In [ ]:
xr_dataset = filtered_img.wx.to_xarray(crs='EPSG:4326', scale=1000)

Requesting data:   0%|          | 0/3 [00:00<?, ?it/s]

Create an automatic pipeline to check the max number of pixels

- limit pixel size to the maximum allowed pixels
- based on it downscale the image as required
- Also try splitting the image into chunks to overcome the maximum limits if exceeding the max pixel request limit

In [ ]:
crs = '''PROJCS["WGS 84 / UTM zone 46N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",93],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32646"]]'''
crs = ee.Projection(crs).getInfo()

In [ ]:
xr_dataset.B1

<xarray.DataArray 'B1' (time: 3, y: 2, x: 3)>
array([[[419.,  nan,  nan],
        [ nan,  nan,  nan]],

       [[ nan, 444.,  nan],
        [ nan,  nan,  nan]],

       [[219.,  nan,  nan],
        [ nan,  nan,  nan]]])
Coordinates:
  * time         (time) datetime64[ns] 2023-01-14T05:41:02 ... 2023-01-19T05:...
  * x            (x) float64 76.5 77.5 78.5
  * y            (y) float64 28.5 27.5
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    _FillValue:              -32768
    scale_factor:            1.0
    add_offset:              0.0

Yes, you can estimate the total number of pixels and the size of an image in Google Earth Engine based on the area of interest (AOI) and the scale (resolution) without actually loading the image. Here’s how you can do it:

1. **Calculate the total number of pixels:**
   - Determine the area of the AOI.
   - Determine the scale (resolution) of the image in meters.

2. **Estimate the total size of the image:**
   - Consider the number of bands in the image.
   - Consider the data type (bit depth) of each pixel.

### Step-by-Step Estimation

#### 1. Calculate the Total Number of Pixels

First, calculate the total area of the AOI in square meters. If you have the AOI as a geometry, you can get its area using Earth Engine:

```javascript
// Define your area of interest (AOI)
var aoi = ee.Geometry.Polygon([
  [[-122.5, 37.7], [-122.5, 37.8], [-122.4, 37.8], [-122.4, 37.7]]
]);

// Calculate the area in square meters
var area = aoi.area();
print('Area in square meters:', area);
```

Next, calculate the number of pixels by dividing the total area by the pixel area (scale squared):

```javascript
// Define the scale in meters
var scale = 30; // for example, 30 meters

// Calculate the number of pixels
var numberOfPixels = area.divide(scale * scale);
print('Number of pixels:', numberOfPixels);
```

#### 2. Estimate the Total Size of the Image

To estimate the size of the image in bytes, consider the number of bands and the bit depth of the image:

```javascript
// Define the number of bands and bit depth
var numberOfBands = 3; // for example, RGB image
var bitDepth = 16; // for example, 16-bit data

// Calculate bytes per pixel
var bytesPerPixel = numberOfBands * (bitDepth / 8);

// Calculate the total size in bytes
var totalSizeBytes = numberOfPixels.multiply(bytesPerPixel);
print('Total size in bytes:', totalSizeBytes);
```

### Example Calculation

Assume:
- AOI: 100 km² (100,000,000 m²)
- Scale: 30 meters
- Bands: 3 (e.g., RGB)
- Bit depth: 16 bits

```javascript
// Define the area in square meters
var area = ee.Number(100000000); // 100 km² in m²

// Define the scale in meters
var scale = 30;

// Calculate the number of pixels
var numberOfPixels = area.divide(scale * scale);

// Define the number of bands and bit depth
var numberOfBands = 3;
var bitDepth = 16;

// Calculate bytes per pixel
var bytesPerPixel = numberOfBands * (bitDepth / 8);

// Calculate the total size in bytes
var totalSizeBytes = numberOfPixels.multiply(bytesPerPixel);

// Print the results
print('Number of pixels:', numberOfPixels);
print('Total size in bytes:', totalSizeBytes);
```

### Running this in Google Earth Engine JavaScript Console

```javascript
// Define your area of interest (AOI)
var aoi = ee.Geometry.Polygon([
  [[-122.5, 37.7], [-122.5, 37.8], [-122.4, 37.8], [-122.4, 37.7]]
]);

// Calculate the area in square meters
var area = aoi.area();
print('Area in square meters:', area);

// Define the scale in meters
var scale = 30; // for example, 30 meters

// Calculate the number of pixels
var numberOfPixels = area.divide(scale * scale);
print('Number of pixels:', numberOfPixels);

// Define the number of bands and bit depth
var numberOfBands = 3; // for example, RGB image
var bitDepth = 16; // for example, 16-bit data

// Calculate bytes per pixel
var bytesPerPixel = numberOfBands * (bitDepth / 8);

// Calculate the total size in bytes
var totalSizeBytes = numberOfPixels.multiply(bytesPerPixel);
print('Total size in bytes:', totalSizeBytes);
```

This script will give you an estimation of the total number of pixels and the size of the image in bytes based on the AOI and scale you provide.